In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import os
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/jovyan/.local/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jovyan/.local/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
year = 2022
month = 2

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
df['pred'] = model.predict(X_val)

In [8]:
ride_id = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result = df[['ride_id', 'pred']]

In [9]:
df_result.to_parquet(
    "data.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [10]:
os.path.getsize("data.parquet")

59994813